In [1]:
import json
import tqdm
import sys

sys.path.append('..')
from preprocess_data import (raw_train_data_file, raw_dev_data_file, raw_test_data_file, processed_dev_data_file, processed_test_data_file, processed_train_data_file, generate_seq)

In [2]:
raw_train_data = json.load(open(raw_train_data_file))['entries']

In [3]:
data_samples = [data_sample[list(data_sample.keys())[0]] for data_sample in raw_train_data]

In [4]:
new_data_samples = []
for data_sample in tqdm.tqdm(data_samples):
    new_data_sample = data_sample.copy()
    unused_labels = ['originaltriplesets', 'xml_id', 'size', 'shape', 'shape_type']
    for label in unused_labels:
        new_data_sample.pop(label)
    triples = new_data_sample['modifiedtripleset']
    new_data_sample['input_seq'], new_data_sample['properties'] = generate_seq(triples)
    new_data_sample['target_sents'] = [sent['lex'] for sent in new_data_sample.pop('lexicalisations')]
    new_data_samples.append(new_data_sample)

100%|██████████| 12876/12876 [00:00<00:00, 53692.30it/s]


In [5]:
new_data_samples[0]

{'category': 'Airport',
 'modifiedtripleset': [{'object': '"Aarhus, Denmark"',
   'property': 'cityServed',
   'subject': 'Aarhus_Airport'}],
 'input_seq': ['Aarhus_Airport', '[P]', '"Aarhus, Denmark"'],
 'properties': ['cityServed'],
 'target_sents': ['The Aarhus is the airport of Aarhus, Denmark.',
  'Aarhus Airport serves the city of Aarhus, Denmark.']}

In [ ]:
json.dumps(new_data_samples[0])

## Check entity

In [2]:
with open(processed_train_data_file) as f_in:
    train_data = [json.loads(line) for line in f_in]
with open(processed_dev_data_file) as f_in:
    dev_data = [json.loads(line) for line in f_in]

In [ ]:
train_data[0]

In [4]:
train_entities_set = set()
train_relation_set = set()
for data in train_data:
    train_entities_set.update([tri['subject'] for tri in data['modifiedtripleset']])
    train_entities_set.update([tri['object'] for tri in data['modifiedtripleset']])
    train_relation_set.update([tri['property'] for tri in data['modifiedtripleset']])

dev_entities_set = set()
dev_relation_set = set()
train_relation_list = []
for data in dev_data:
    dev_entities_set.update([tri['subject'] for tri in data['modifiedtripleset']])
    dev_entities_set.update([tri['object'] for tri in data['modifiedtripleset']])
    dev_relation_set.update([tri['property'] for tri in data['modifiedtripleset']])
    train_relation_list += [tri['property'] for tri in data['modifiedtripleset']]

from collections import Counter

c = Counter(train_relation_list)
print(len(train_entities_set))
print(len(dev_entities_set))
uncovered_entities_set = dev_entities_set - train_entities_set
print(len(uncovered_entities_set))
print('')
print(len(train_relation_set))
print(len(dev_relation_set))
uncovered_relation_set = dev_relation_set - train_relation_set
print(len(uncovered_relation_set))

3105
2003
19

373
288
0


In [5]:
c.most_common(20)

[('country', 349),
 ('leaderName', 207),
 ('location', 195),
 ('birthPlace', 194),
 ('isPartOf', 185),
 ('club', 138),
 ('ethnicGroup', 112),
 ('associatedBand/associatedMusicalArtist', 109),
 ('language', 97),
 ('genre', 94),
 ('region', 86),
 ('deathPlace', 84),
 ('ingredient', 83),
 ('capital', 82),
 ('ground', 53),
 ('battles', 53),
 ('manager', 48),
 ('nationality', 46),
 ('birthDate', 45),
 ('almaMater', 45)]